In [1]:
import numpy as np
import pandas as pd
import random
import warnings

# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

**Пути к директориям и папкам**

In [2]:
TRAIN_DATASET_PATH = 'main_table.xlsx'

# Загрузка данных

**Описание датасета:**

В датасете представлены данные о доходах сотрудников участка горячего проката за год, в том числе данные, описывающие специфику рабочих мест  

**Задача:**  

* Выявить формулу дохода в целом и его составных частей  
* Проверить, что данная формула объясняет доход с точностью до рубля  
* Выявить зависимости между составными частями дохода  

# Предварительный анализ
**Для начала посмотрим на данные**

In [3]:
train_df = pd.read_excel(TRAIN_DATASET_PATH)
train_df.head()

,FIO,Месяц,Профессия,Разряд,Оборудование,Вредность (балл.),Объем производства,Доход
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588


In [4]:
#заменим названия столбцов
train_df.columns = ['name', 'month', 'profession', 'grade', 'equipment',
       'harmfulness', 'production_volume', 'income']
#приведём дату к формату даты
train_df['month'] = pd.to_datetime(train_df['month'], '%Y-%m-%d')
#проверим результат
train_df.head()

,name,month,profession,grade,equipment,harmfulness,production_volume,income
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588


In [5]:
train_df.shape

(2400, 8)

In [6]:
#посмотрим на количество уникальных значений
train_df.nunique()

name                 200
month                 12
profession             9
grade                  5
equipment              9
harmfulness            6
production_volume     12
income               346
dtype: int64

**Первые выводы:**  

* **1)** Уникальное количество месяцев и объём производимой продукции равен из чего можно сделать вывод, что объём производимой продукции указан в общем для всего цеха, а не уникален для каждого рабочего 
* **2)** Всего 200 сотрудников на производстве, работающих на 9 разных профессиях, с 9 разным оборудованием и 6 показателями вредности

# Обзор данных
**Целевая переменная (income)**  
Рассмотрим средние значения на основе разных признаков, чтобы выявить некоторые закономерности

In [7]:
train_df.groupby('month')['income'].mean()

month
2009-01-01    19885.912500
2009-02-01    19885.912500
2009-03-01    19885.912500
2009-04-01    23395.097426
2009-05-01    23896.409559
2009-06-01    20888.536765
2009-07-01    25400.345956
2009-08-01    23194.572574
2009-09-01    22191.948309
2009-10-01    19885.912500
2009-11-01    21590.373750
2009-12-01    19885.912500
Name: income, dtype: float64

Самый высокий доход сотрудников оказался в 7 месяце

In [8]:
#проверим наличие зависимости от объёма производства
train_df.groupby('month')['production_volume'].mean()

month
2009-01-01     580
2009-02-01     790
2009-03-01     820
2009-04-01    1200
2009-05-01    1250
2009-06-01     950
2009-07-01    1400
2009-08-01    1180
2009-09-01    1080
2009-10-01     780
2009-11-01    1020
2009-12-01     810
Name: production_volume, dtype: int64

In [9]:
train_df.groupby('production_volume')['income'].mean()

production_volume
580     19885.912500
780     19885.912500
790     19885.912500
810     19885.912500
820     19885.912500
950     20888.536765
1020    21590.373750
1080    22191.948309
1180    23194.572574
1200    23395.097426
1250    23896.409559
1400    25400.345956
Name: income, dtype: float64

Действительно самый высокий доход сотрудников в 7 месяце совпадает с самым высоким показателем объёма производства  
<span style="color:red">Чем выше объём производства, тем выше доход</span>

In [10]:
train_df.groupby('profession')['income'].mean()

profession
Вальцовщик                              25744.890874
Контролер качества готовой продукции    25175.434314
Нагревальщик металла                    24282.386064
Огнеупорщик                             18053.498382
Оператор поста управления               18929.055664
Подручный вальцовщика                   18447.473897
Посадчик металла                        20150.538419
Резчик горячего металла                 18708.271639
Уборщик                                 15559.387395
Name: income, dtype: float64

In [11]:
train_df['profession'].value_counts() / 12

Вальцовщик                              51.0
Нагревальщик металла                    42.0
Подручный вальцовщика                   30.0
Оператор поста управления               27.0
Уборщик                                 21.0
Контролер качества готовой продукции     9.0
Посадчик металла                         8.0
Резчик горячего металла                  7.0
Огнеупорщик                              5.0
Name: profession, dtype: float64

Самый низкий доход у уборщика, самый высокий у вальцовщика  
<span style="color:red">Профессий вальцовщик больше всего, они имеют пограничное значение дохода, поэтому рассмотрение стоит начать именно с них</span>

In [12]:
train_df.groupby('grade')['income'].mean()

grade
3    16242.927731
4    18894.771704
5    21687.454268
6    25208.128529
7    27923.024923
Name: income, dtype: float64

<span style="color:red">Доход напрямую зависит от разряда сотрудника, чем выше разряд тем выше доход</span>

In [13]:
train_df.groupby('equipment')['income'].mean()

equipment
Вспомогательные помещения              13750.662745
Калибровочный стан                     19501.114255
Калибровочный стан, ГРП                20748.062745
Контрольно-измерительная аппаратура    25175.434314
Нагревательные печи                    23348.397620
Нагревательные печи, кантовка          22400.708364
Нагревательные печи, нагрев            22765.248611
Пилигримовые станы                     20758.343058
Прошивной стан                         24441.357021
Name: income, dtype: float64

In [14]:
train_df.groupby('harmfulness')['income'].mean()

harmfulness
0     19463.048529
8     18929.055664
11    18053.498382
16    18708.271639
18    22251.664753
22    23621.290441
Name: income, dtype: float64

<span style="color:red">Доход имеет НЕ линейную зависимость от вредности производства</span>

In [15]:
train_df.loc[train_df['harmfulness'] == 0].head()

,name,month,profession,grade,equipment,harmfulness,production_volume,income
96,Антонов В. Т.,2009-01-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,580,22060.000000
97,Антонов В. Т.,2009-02-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,790,22060.000000
98,Антонов В. Т.,2009-03-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,820,22060.000000
99,Антонов В. Т.,2009-04-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,1200,26601.764706
100,Антонов В. Т.,2009-05-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,1250,27250.588235


<span style="color:red">Нулевой уровень вредности имеют только "Контролер качества готовой продукции" и "Уборщик" вспомогательных помещений</span>

In [16]:
train_df.loc[train_df['equipment'] == 'Контрольно-измерительная аппаратура'].head()

,name,month,profession,grade,equipment,harmfulness,production_volume,income
96,Антонов В. Т.,2009-01-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,580,22060.000000
97,Антонов В. Т.,2009-02-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,790,22060.000000
98,Антонов В. Т.,2009-03-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,820,22060.000000
99,Антонов В. Т.,2009-04-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,1200,26601.764706
100,Антонов В. Т.,2009-05-01,Контролер качества готовой продукции,7,Контрольно-измерительная аппаратура,0,1250,27250.588235


<span style="color:red">Абсолютно все Контролер качества готовой продукции работают с нулевой вредностью производства</span>

# Продолжаем поиск закономерностей

# Рассчёт уровня плана

In [17]:
train_df.loc[(train_df['profession'] == 'Уборщик') & (train_df['equipment'] == 'Вспомогательные помещения') ]

,name,month,profession,grade,equipment,harmfulness,production_volume,income
468,Григорьев А. Т.,2009-01-01,Уборщик,3,Вспомогательные помещения,0,580,12180.000000
469,Григорьев А. Т.,2009-02-01,Уборщик,3,Вспомогательные помещения,0,790,12180.000000
470,Григорьев А. Т.,2009-03-01,Уборщик,3,Вспомогательные помещения,0,820,12180.000000
471,Григорьев А. Т.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059
472,Григорьев А. Т.,2009-05-01,Уборщик,3,Вспомогательные помещения,0,1250,15045.882353
473,Григорьев А. Т.,2009-06-01,Уборщик,3,Вспомогательные помещения,0,950,12896.470588
474,Григорьев А. Т.,2009-07-01,Уборщик,3,Вспомогательные помещения,0,1400,16120.588235
475,Григорьев А. Т.,2009-08-01,Уборщик,3,Вспомогательные помещения,0,1180,14544.352941
476,Григорьев А. Т.,2009-09-01,Уборщик,3,Вспомогательные помещения,0,1080,13827.882353
477,Григорьев А. Т.,2009-10-01,Уборщик,3,Вспомогательные помещения,0,780,12180.000000


Уборщик в Вспомогательные помещения имеет самый низкий grade и нулевой harmfulness, постараемся определить уровень плана на производстве, после которого начинается доплата за перевыполнение плана

In [18]:
(16120.59 - 15045.88 )/ (1400 - 1250)

7.16473333333334

In [19]:
(16120.59- 14544.35)/(1400 - 1180)

7.164727272727272

In [20]:
(16120.59 - 12180.00)/ (1400 - 850)

7.164709090909091

План равен 850

In [21]:
#перепроверим соответствие на других сотрудниках
train_df.loc[(train_df['profession'] == 'Уборщик') & (train_df['equipment'] == 'Пилигримовые станы') ]

,name,month,profession,grade,equipment,harmfulness,production_volume,income
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588
5,Абрамов Е. Д.,2009-06-01,Уборщик,3,Пилигримовые станы,18,950,17426.117647
6,Абрамов Е. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20972.647059
7,Абрамов Е. Д.,2009-08-01,Уборщик,3,Пилигримовые станы,18,1180,19238.788235
8,Абрамов Е. Д.,2009-09-01,Уборщик,3,Пилигримовые станы,18,1080,18450.670588
9,Абрамов Е. Д.,2009-10-01,Уборщик,3,Пилигримовые станы,18,780,16638.000000


In [22]:
(20972.65-16638.00) / (1400 - 850)

7.881181818181821

In [23]:
(20972.65-19238.79) / (1400 - 1180)

7.881181818181821

In [24]:
#перепроверим соответствие на других сотрудниках
train_df.loc[(train_df['profession'] == 'Вальцовщик') & (train_df['equipment'] == 'Пилигримовые станы')]

,name,month,profession,grade,equipment,harmfulness,production_volume,income
24,Агафонов С. И.,2009-01-01,Вальцовщик,6,Пилигримовые станы,18,580,22260.000000
25,Агафонов С. И.,2009-02-01,Вальцовщик,6,Пилигримовые станы,18,790,22260.000000
26,Агафонов С. И.,2009-03-01,Вальцовщик,6,Пилигримовые станы,18,820,22260.000000
27,Агафонов С. И.,2009-04-01,Вальцовщик,6,Пилигримовые станы,18,1200,26175.882353
28,Агафонов С. И.,2009-05-01,Вальцовщик,6,Пилигримовые станы,18,1250,26735.294118
29,Агафонов С. И.,2009-06-01,Вальцовщик,6,Пилигримовые станы,18,950,23378.823529
30,Агафонов С. И.,2009-07-01,Вальцовщик,6,Пилигримовые станы,18,1400,28413.529412
31,Агафонов С. И.,2009-08-01,Вальцовщик,6,Пилигримовые станы,18,1180,25952.117647
32,Агафонов С. И.,2009-09-01,Вальцовщик,6,Пилигримовые станы,18,1080,24833.294118
33,Агафонов С. И.,2009-10-01,Вальцовщик,6,Пилигримовые станы,18,780,22260.000000


In [25]:
(28413.53 - 22260.00) / (1400-850)

11.188236363636362

In [26]:
(28413.53 - 24833.29) / (1400 - 1080)

11.188249999999993

<span style="color:red">Уровень плана, после которого начинается премия найден и равен 850</span>

In [27]:
#добавим новый столбец процент перевыполнения плана
train_df['over_plan_volume'] = train_df['production_volume'] - 850
train_df.loc[(train_df['over_plan_volume'] < 0), 'over_plan_volume'] = 0
train_df.head()

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000,0
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000,0
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000,0
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765,350
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588,400


In [28]:
train_df.loc[(train_df['profession'] == 'Уборщик')]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000,0
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000,0
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000,0
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765,350
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588,400
5,Абрамов Е. Д.,2009-06-01,Уборщик,3,Пилигримовые станы,18,950,17426.117647,100
6,Абрамов Е. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20972.647059,550
7,Абрамов Е. Д.,2009-08-01,Уборщик,3,Пилигримовые станы,18,1180,19238.788235,330
8,Абрамов Е. Д.,2009-09-01,Уборщик,3,Пилигримовые станы,18,1080,18450.670588,230
9,Абрамов Е. Д.,2009-10-01,Уборщик,3,Пилигримовые станы,18,780,16638.000000,0


# Рассчёт доплаты за вредность производства

In [29]:
#посмотрим на различия у уборщиков (всего 2 варианта: Вспомогательные помещения + 0 и Пилигримовые станы + 18)
train_df.loc[(train_df['profession'] == 'Уборщик') & (train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.0,0
12,Абрамов Ф. М.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16029.0,0
48,Александров А. М.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,15420.0,0
384,Воронов Р. И.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,15420.0,0
468,Григорьев А. Т.,2009-01-01,Уборщик,3,Вспомогательные помещения,0,580,12180.0,0
612,Журавлев Б. И.,2009-01-01,Уборщик,3,Вспомогательные помещения,0,580,12180.0,0
792,Клюев Д. М.,2009-01-01,Уборщик,3,Вспомогательные помещения,0,580,12180.0,0
864,Комаров А. И.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,15420.0,0
912,Кондратьев Д. Д.,2009-01-01,Уборщик,3,Вспомогательные помещения,0,580,12789.0,0
984,Краснов И. М.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,15420.0,0


In [30]:
#посчитаем доплату за вредность производства
(15420.00 - 12180.00)/18

180.0

In [31]:
#перепроверим соответствие на других сотрудниках
train_df.loc[(train_df['grade'] == 5) & (train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
60,Алексеев Н. Н.,2009-01-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,580,20350.0,0
72,Андреев Е. А.,2009-01-01,Вальцовщик,5,Калибровочный стан,18,580,20449.5,0
84,Андреев Ф. Д.,2009-01-01,Оператор поста управления,5,Калибровочный стан,8,580,18649.5,0
108,Баженов Д. И.,2009-01-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,580,18649.5,0
240,Васильев С. В.,2009-01-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,580,20350.0,0
252,Вдовин И. К.,2009-01-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,580,19469.0,0
288,Винокуров М. Т.,2009-01-01,Вальцовщик,5,Калибровочный стан,18,580,20449.5,0
300,Власов В. Д.,2009-01-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,580,21989.0,0
372,Воробьев Н. П.,2009-01-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,580,20350.0,0
396,Высоцкий М. А.,2009-01-01,Нагревальщик металла,5,"Нагревательные печи, нагрев",22,580,20350.0,0


In [32]:
(20350.00 - 17830.00)/(22-8)

180.0

<span style="color:red">Найдена доплата за 1ед вредности производства 180. Так же сделан вывод о том, что используемое сотрудником оборудование не влияет на выплаты, а так же профессия</span>

# Рассчёт доплаты за разряд

In [33]:
#посчитаем разницу в выплатах вальцовщиков 5, 6 и 7 разрядов
train_df.loc[(train_df['profession'] == 'Вальцовщик') & (train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
24,Агафонов С. И.,2009-01-01,Вальцовщик,6,Пилигримовые станы,18,580,22260.0,0
72,Андреев Е. А.,2009-01-01,Вальцовщик,5,Калибровочный стан,18,580,20449.5,0
120,Белкин В. И.,2009-01-01,Вальцовщик,7,Пилигримовые станы,18,580,25300.0,0
168,Белоусов М. Д.,2009-01-01,Вальцовщик,6,Пилигримовые станы,18,580,23211.0,0
192,Богданов А. Т.,2009-01-01,Вальцовщик,7,Прошивной стан,18,580,26403.0,0
264,Верещагин Я. А.,2009-01-01,Вальцовщик,7,Прошивной стан,18,580,26403.0,0
288,Винокуров М. Т.,2009-01-01,Вальцовщик,5,Калибровочный стан,18,580,20449.5,0
312,Власов С. Д.,2009-01-01,Вальцовщик,7,Прошивной стан,18,580,25300.0,0
324,Волков А. А.,2009-01-01,Вальцовщик,7,Прошивной стан,18,580,25300.0,0
408,Гаврилов Д. Е.,2009-01-01,Вальцовщик,7,Прошивной стан,18,580,26403.0,0


In [34]:
#посчитаем разницу между 5 и 6 разрядом
(22260.00 - (180*18))/( 19630.00 - (180*18))

1.160463697376449

In [35]:
#посчитаем разницу между 6 и 7 разрядом
(25300.00 - (180*18))/( 22260.00 - (180*18))

1.159831756046267

In [36]:
#посчитаем разницу между 5 и 7 разрядом
(25300.00 - (180*18))/( 19630.00 - (180*18))

1.3459426479560708

In [37]:
import math
math.sqrt (1.3459426479560708) 

1.1601476836834486

<span style="color:red">Коэффициент доплаты за разряд равен 1.16 от предыдущего</span>

# Рассчёт доплаты за перевыполнение плана

In [38]:
#посчитаем доплату за перевыполнение плана для 3 разряда
train_df.loc[(train_df['profession'] == 'Уборщик')& (train_df['harmfulness'] == 0) & (train_df['production_volume'] == 1200)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
471,Григорьев А. Т.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350
615,Журавлев Б. И.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350
795,Клюев Д. М.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350
915,Кондратьев Д. Д.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,15422.029412,350
1215,Львов А. С.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350
1371,Наумов О. В.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350
2007,Степанов А. М.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,15422.029412,350
2151,Федотов М. М.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,16156.411765,350
2295,Чернышев Д. В.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687.647059,350


In [39]:
#доплата для 3 разряда
(14687.65 - 12180.00)/350

7.164714285714284

In [40]:
#посчитаем доплату за перевыполнение плана для 4 разряда
train_df.loc[(train_df['grade'] == 4) &  (train_df['harmfulness'] == 8) &(train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
276,Виноградов П. Д.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
552,Емельянов Е. Н.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
744,Калмыков А. Ф.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
924,Константинов П. Г.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,16276.5,0
1416,Никулин Е. А.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
1488,Носов М. Д.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,16276.5,0
1524,Орлов В. В.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
1704,Романов Р. К.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
1860,Ситников Н. М.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0
1968,Сорокин К. Т.,2009-01-01,Оператор поста управления,4,Калибровочный стан,8,580,15570.0,0


In [41]:
#доплата для 4 разряда
(18479.12 - 15570.00)/350

8.311771428571426

In [42]:
8.311771428571426 / 7.164714285714284

1.1600980998145674

In [43]:
#доплата для 5 разряда
#посчитаем доплату за перевыполнение плана для 4 разряда
train_df.loc[(train_df['grade'] == 5) &  (train_df['harmfulness'] == 8) &(train_df['production_volume'] == 1200)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
87,Андреев Ф. Д.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,22192.632353,350
111,Баженов Д. И.,2009-04-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,1200,22192.632353,350
255,Вдовин И. К.,2009-04-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,1200,23180.852941,350
423,Герасимов К. Я.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,21204.411765,350
627,Зайцев Э. А.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,22192.632353,350
879,Комаров Д. А.,2009-04-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,1200,23180.852941,350
891,Комаров Д. К.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,21204.411765,350
963,Корчагин М. Г.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,22192.632353,350
1107,Лазарев А. Д.,2009-04-01,Оператор поста управления,5,Калибровочный стан,8,1200,23180.852941,350
1299,Михайлов Р. С.,2009-04-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,1200,23180.852941,350


In [44]:
#доплата для 5 разряда
(21204.41 - 17830.00)/350

9.641171428571429

In [45]:
9.641171428571429 / 7.164714285714284

1.3456463222539035

In [46]:
import math
math.sqrt (1.3459426479560708) 

1.1601476836834486

<span style="color:red">Коэффициент доплаты за перевыполнение плана равен 1.16 от предыдущего разряда умноженный на базовый коэффециент 7.164714285714284</span>

# Рассчёт индивидуальных доплат

In [47]:
#посчитаем премии
train_df.loc[(train_df['grade'] == 4) &  (train_df['harmfulness'] == 22) &(train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
432,Голиков Г. И.,2009-01-01,Нагревальщик металла,4,Нагревательные печи,22,580,18796.5,0
540,Егоров Г. И.,2009-01-01,Посадчик металла,4,Нагревательные печи,22,580,18090.0,0
636,Захаров К. Д.,2009-01-01,Посадчик металла,4,Нагревательные печи,22,580,19503.0,0
708,Ильин Э. П.,2009-01-01,Нагревальщик металла,4,Нагревательные печи,22,580,18090.0,0
720,Ильинский Н. Д.,2009-01-01,Посадчик металла,4,Нагревательные печи,22,580,18090.0,0
852,Колпаков И. А.,2009-01-01,Нагревальщик металла,4,Нагревательные печи,22,580,18090.0,0
1056,Куликов А. Л.,2009-01-01,Посадчик металла,4,Нагревательные печи,22,580,19503.0,0
1236,Максимов И. В.,2009-01-01,Посадчик металла,4,Нагревательные печи,22,580,18796.5,0
1284,Михайлов М. М.,2009-01-01,Нагревальщик металла,4,Нагревательные печи,22,580,19503.0,0
1380,Некрасов И. А.,2009-01-01,Нагревальщик металла,4,Нагревательные печи,22,580,18796.5,0


In [48]:
#посчитаем индивидуальную доплату 1
(18796.50- (180 * 22))/(18090.00 - (180 * 22))

1.05

In [49]:
#посчитаем индивидуальную доплату 2
(19503.00- (180 * 22))/(18090.00 - (180 * 22))

1.1

In [50]:
#проверим результат на других профессиях
train_df.loc[(train_df['grade'] == 5) &  (train_df['harmfulness'] == 22) &(train_df['production_volume'] == 1200)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
63,Алексеев Н. Н.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,23724.411765,350
243,Васильев С. В.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,23724.411765,350
303,Власов В. Д.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,25700.852941,350
375,Воробьев Н. П.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,23724.411765,350
399,Высоцкий М. А.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, нагрев",22,1200,23724.411765,350
591,Ефимов В. Н.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, нагрев",22,1200,24712.632353,350
687,Игнатьев В. Г.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,24712.632353,350
699,Ильин К. Д.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,23724.411765,350
759,Карпов А. М.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, нагрев",22,1200,24712.632353,350
1011,Кузнецов А. С.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, нагрев",22,1200,25700.852941,350


In [51]:
#посчитаем индивидуальную доплату 1
(24712.63 - (180 * 22))/(23724.41 - (180 * 22))

1.0499999747020023

In [52]:
#посчитаем индивидуальную доплату 2
(25700.85 - (180 * 22))/(23724.41 - (180 * 22))

1.0999999494040045

<span style="color:red">Индивидуальная премия равно 5% либо 10% от дохода без учёта доплат за вредность</span>

# Рассчёт готовой формулы

Общий доход равен базовом окладу в 12180 помноженному на коэффициент доплат за каждый разряд   
плюс   
премия за переработку на коэффициент доплат за каждый разряд  
плюс  
доплата за каждую единицу вредности производства

In [53]:
def income_count(grade, over_plan_volume, individual_income_percent, harmfulness):
    income = 1.16 ** (grade - 3) * (12180 + (7.164714285714284 * over_plan_volume))* (1 + (individual_income_percent/100)) + harmfulness * 180
    print (income)

**grade** - разряд (число от 3 до 7)  
**over_plan_value** - (объём производства выше плана (план 850))  
**individual_income_percent** - ставка индивидуальных доплат (0,5,10 процентов)  
**harmfulness** - уровень вредности  

# Проверим формулу

In [54]:
train_df.loc[(train_df['name'] == 'Лазарев Д. А.') &(train_df['production_volume'] == 1200)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
1131,Лазарев Д. А.,2009-04-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1200,23724.411765,350


In [55]:
income_count(5,350,0,22)

23723.701839999998


In [56]:
train_df.loc[(train_df['name'] == 'Комаров Д. А.') &(train_df['production_volume'] == 580)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
876,Комаров Д. А.,2009-01-01,Оператор поста управления,5,"Калибровочный стан, ГРП",8,580,19469.0,0


In [57]:
income_count(5,0,10,8)

19468.3488


In [58]:
train_df.loc[(train_df['name'] == 'Потапов В. М.') &(train_df['production_volume'] == 1250)]

,name,month,profession,grade,equipment,harmfulness,production_volume,income,over_plan_volume
1672,Потапов В. М.,2009-05-01,Нагревальщик металла,5,"Нагревательные печи, кантовка",22,1250,24206.470588,400


In [59]:
income_count(5,400,0,22)

24205.743817142855


# Рассчёт формулы закончен